## The Topaz pipeline

The Topaz particle picking pipeline proceeds as follows:

(Before running Topaz, optional): label a small (100-1000, more is likely to give better results) number of particles on your micrographs using the software of your choice. This can be skipped when using the pretrained models bundled with Topaz. 

1. (Preprocessing) Micrographs are downsampled and normalized, any labeled particle coordinates also need to be scaled appropriately
2. (Training, optional) The particle detection model is trained on the preprocessed micrographs using the labeled particle coordinates. This requires setting the expected number of particles per micrograph. This can be skipped when using the pretrained models bundled with Topaz.
3. (Extraction) Using a trained model, particle coordinates and their associated scores are extracted from the micrographs. This requires knowing the particle radius in pixels on the downsampled micrographs.

(Optional postprocessing): examine classifier performance, rescale particle coordinates, extract particle stack, change particle file format, filter particles by model score, etc.

### In this guide

We assume that the user already has a file containing their labeled particle coordinates (data/EMPIAR-10025/rawdata/particles.txt in this case), that the expected number of particles per micrograph is 300, and that the particle radius is ~74 Å (14 pixels after downsampling 8x).

# 1. Preprocessing

Downsample the micrographs by a factor of 8 (to ~5.28 Å/pix) and also scale the labeled coordinates to match.

__A note on downsampling:__ For this demo dataset, we downsample 8x, but this may not always be the best downsampling amount for your data. We recommend downsampling your data enough that the diameter of your particle fits within the receptive field of the CNN architecture you are using. In this tutorial, we use the default architecture of resnet8, which has a receptive field size of 71 pixels. If you are using a different architecture, this number is slightly different. See [here](https://github.com/tbepler/topaz#model-architectures) for architecture details.

As a rule of thumb, downsampling to about 4-8 Å per pixel works well, but this may need to be adjusted for very large or very small particles to fit the classifier as described above.

__Pretrained models:__ When using the pretrained models, they assume that microcraphs are downsampled to the 4-8 Å per pixel range. Generally speaking, ~4 Å/pix is better for small to medium sized particles, but ~8 Å/pix is better for large particles (>= 300 Å particle diameter). Very large particles may require additional downsampling.

In [1]:
import sys
import importlib
import topaz
from topaz.main import main

# importlib.reload(topaz.main)
import pdb
main?

# replace /miniconda3/envs/topaz/lib/python3.12/site-packages/topaz/main.py - DONE
# figure out softlinking so that my repo changes lead to changes in the miniconda package

Signature: main(cmd=None)
Docstring: this is main!
File:      ~/miniconda3/envs/topaz/lib/python3.12/site-packages/topaz/main.py
Type:      function

In [17]:
# %debug
import glob
input_files = glob.glob('/h2/kspivakovsky/topaz_tutorial_data/EMPIAR-10025/rawdata/micrographs/*.mrc')
input_files = ' '.join(input_files)

# create command
cmd = f'''preprocess -s 8 -o{input_files}'''
# cmd = f'''preprocess -s 8 -d -1 -o /fastData/topaz_tutorial_data/EMPIAR-10025/processed/m2/ {input_files}'''

#topaz.main.main(cmd)
# import pdb;breakpoint()
# main(cmd)
# topaz.main.main(cmd)
pdb.run(f'''topaz.main.main(cmd)''')

> <string>(1)<module>()



ipdb>  s


--Call--
> /h2/kspivakovsky/miniconda3/envs/topaz/lib/python3.12/site-packages/topaz/main.py(53)main()
     51 
     52 
---> 53 def main(cmd=None):
     54     '''this is main!'''
     55     import argparse



ipdb>  q


In [11]:
%%bash
# source activate topaz

# we'll store the processed data in data/EMPIAR-10025/processed
# so we need to make these directories first
# mkdir -p data/EMPIAR-10025/processed
# mkdir -p data/EMPIAR-10025/processed/micrographs

# to run the preprocess command, we pass the input micrographs as command line arguments
# preprocess will write the processed images to the directory specified with the -o argument
# -s sets the downsampling amount (in this case, we downsample by a factor of 8)
# -d/--device X sets preprocess to use GPU with ID X
# -t/--num-workers X sets preprocess to use X processes (preprocesses X micrographs in parallel), this and GPU device are mutually exclusive
# -v gives verbose output to track progress
topaz preprocess -v -s 8 -o /h2/kspivakovsky/topaz_tutorial_data/EMPIAR-10025/processed/micrographs/ /h2/kspivakovsky/topaz_tutorial_data/EMPIAR-10025/rawdata/micrographs/*.mrc

# this command takes the particle coordinates matched to the original micrographs
# and scales them by 1/8 (-s is downscaling)
# the -x option applies upscaling instead
topaz convert -s 8 -o /h2/kspivakovsky/topaz_tutorial_data/EMPIAR-10025/processed/particles.txt /h2/kspivakovsky/topaz_tutorial_data/EMPIAR-10025/rawdata/particles.txt

# processed: 14sep05c_c_00003gr_00014sq_00004hl_00004es_c
# processed: 14sep05c_c_00003gr_00014sq_00005hl_00003es_c
# processed: 14sep05c_c_00003gr_00014sq_00007hl_00004es_c
# processed: 14sep05c_c_00003gr_00014sq_00011hl_00003es_c
# processed: 14sep05c_c_00003gr_00015sq_00015hl_00002es_c
# processed: 14sep05c_c_00003gr_00018sq_00008hl_00003es_c
# processed: 14sep05c_c_00003gr_00018sq_00010hl_00005es_c
# processed: 14sep05c_c_00003gr_00020sq_00011hl_00002es_c
# processed: 14sep05c_c_00003gr_00020sq_00011hl_00004es_c
# processed: 14sep05c_c_00004gr_00031sq_00002hl_00002es_c
# processed: 14sep05c_c_00004gr_00031sq_00005hl_00002es_c
# processed: 14sep05c_c_00004gr_00031sq_00010hl_00002es_c
# processed: 14sep05c_c_00004gr_00032sq_00007hl_00003es_c
# processed: 14sep05c_c_00004gr_00032sq_00010hl_00003es_c
# processed: 14sep05c_c_00004gr_00032sq_00029hl_00005es_c
# processed: 14sep05c_c_00004gr_00032sq_00031hl_00002es_c
# processed: 14sep05c_c_00004gr_00032sq_00033hl_00005es_c
# processed: 1

# 2. Model training

Given the preprocessed micrographs and particle coordinates, we train the particle detection model using positive-unlabeled learning. This requires us to specify the expected number of particles per micrograph, which we set to 400 in this case.

In [2]:
%%bash
source activate topaz

# first, make sure we have the folders where we want to put the saved models
# store the saved models in saved_models/EMPIAR-10025
mkdir -p saved_models
mkdir -p saved_models/EMPIAR-10025

# Now, we train the model

# We set -n 400 to tell Topaz that we expect there to be on average 400 particles per micrograph
# and --num-workers=8 to speed up training

# By default, topaz train will use your first GPU if available, to force topaz train to use the CPU, set: -d -1
# To use a different GPU, set -d X where X is the GPU ID

# the models will be saved to the saved_models/EMPIAR-10025 directory
topaz train -n 400 \
            --num-workers=8 \
            --train-images data/EMPIAR-10025/processed/micrographs/ \
            --train-targets data/EMPIAR-10025/processed/particles.txt \
            --save-prefix=saved_models/EMPIAR-10025/model \
            -o saved_models/EMPIAR-10025/model_training.txt

# Loading model: resnet8
# Model parameters: units=32, dropout=0.0, bn=on
# Loading pretrained model: resnet8_u32
# Receptive field: 71
# Using device=0 with cuda=True
# Loaded 30 training micrographs with 1500 labeled particles
# source	split	p_observed	num_positive_regions	total_regions
# 0	train	0.00163	43500	26669790
# Specified expected number of particle per micrograph = 400.0
# With radius = 3
# Setting pi = 0.0130484716977524
# minibatch_size=256, epoch_size=1000, num_epochs=10
# Done!


# 3. Extract particle coordinates

Now that we have a trained model, we use it to extract predicted particle coordinates using a particle radius of 14 pixels.

Extract can be run using a particle picking model trained above by passing the model path as an argument. It can also use the pretrained models by specifying on of:

- resnet8_u32
- resnet8_u64
- resnet16_u32
- resnet16_u64 (the default)

as the model argument. If no argument is passed, extract uses the resnet16_u64 pretrained model.

In [3]:
%%bash
source activate topaz

## make a directory to write the topaz particles to
mkdir -p data/EMPIAR-10025/topaz

## extract particle coordinates using the  trained model
## we set the radius parameter to 14 (-r 14)
## to prevent extracting particle coordinates closer than the radius of the particle
## i.e. we don't want multiple predictions for a single particle
## we also set -x 8 in order to scale the coordinates back to the original micrograph size

topaz extract -r 14 -x 8 -m saved_models/EMPIAR-10025/model_epoch10.sav \
              -o data/EMPIAR-10025/topaz/predicted_particles_all_upsampled.txt \
              data/EMPIAR-10025/processed/micrographs/*.mrc
              
              
## To use the pretrained particle picking model instead, we can omit the model argument.

#topaz extract -r 14 -x 8 \
#              -o data/EMPIAR-10025/topaz/predicted_particles_all_upsampled.txt \
#              data/EMPIAR-10025/processed/micrographs/*.mrc

# (Optional) change format of particle coordinates file

In [4]:
%%bash
source activate topaz

# we can convert the particles file to .star format (and others) by changing the file extension
# of the output file (data/EMPIAR-10025/topaz/predicted_particles_all_upsampled.txt)
# to .star (data/EMPIAR-10025/topaz/predicted_particles_all_upsampled.star)
# the convert command can also filter the particle table by model score using the -t argument
# e.g. -t 0 would only keep particles with scores >= 0

topaz convert -o data/EMPIAR-10025/topaz/predicted_particles_all_upsampled.star \
              data/EMPIAR-10025/topaz/predicted_particles_all_upsampled.txt

# That's it!

We now have a table containing particle coordinates for each micrograph with their corresponding model score.

## Next steps

- To better understand the outputs of the individual steps and to visualize intermediate results, take a look at the detailed walkthrough [here](https://github.com/tbepler/topaz/blob/master/tutorial/02_walkthrough.ipynb)

- To jump straight to understanding model selection and evaluation criteria, take a look at the cross validation tutorial [here](https://github.com/tbepler/topaz/blob/master/tutorial/03_cross_validation.ipynb) 